In [2]:
pwd

'/home/ubuntu/project/notebook'

# Librispeech

In [1]:
!mkdir /home/ubuntu/project/manifest_libri
!mkdir /home/ubuntu/project/manifest_libri/train
!mkdir /home/ubuntu/project/manifest_libri/dev

mkdir: cannot create directory ‘/home/ubuntu/project/manifest_libri’: File exists
mkdir: cannot create directory ‘/home/ubuntu/project/manifest_libri/train’: File exists
mkdir: cannot create directory ‘/home/ubuntu/project/manifest_libri/dev’: File exists


In [5]:
%cd /home/ubuntu/fairseq

/home/ubuntu/fairseq


In [6]:
!python examples/wav2vec/wav2vec_manifest.py "/home/ubuntu/data/LibriSpeech/train-clean-100" \
--dest "/home/ubuntu/project/manifest_libri/train" --ext "flac" --valid-percent 0

In [7]:
!python examples/wav2vec/wav2vec_manifest.py "/home/ubuntu/data/LibriSpeech/dev-clean" \
--dest "/home/ubuntu/project/manifest/dev/" --ext "flac" --valid-percent 1

## Letter dictionary

In [12]:
%cd /home/ubuntu/project/manifest/train/
!wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/dict.ltr.txt

/home/ubuntu/project/manifest/train
--2020-12-04 05:54:40--  https://dl.fbaipublicfiles.com/fairseq/wav2vec/dict.ltr.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207 [text/plain]
Saving to: ‘dict.ltr.txt.1’

dict.ltr.txt.1      100%[===================>]     207  --.-KB/s    in 0s      

2020-12-04 05:54:41 (48.6 MB/s) - ‘dict.ltr.txt.1’ saved [207/207]



## create labels

In [14]:
%cd /home/ubuntu/fairseq/examples/wav2vec
!python libri_labels.py /home/ubuntu/project/manifest_libri/train/train.tsv \
--output-dir /home/ubuntu/project/manifest/train/ --output-name train

/home/ubuntu/fairseq/examples/wav2vec


In [15]:
!mv /home/ubuntu/project/manifest_libri/dev/valid.tsv /home/ubuntu/project/manifest_libri/train

mv: cannot stat '/home/ubuntu/project/manifest_libri/dev/valid.tsv': No such file or directory


# WSJ

## transfer scp files to real wav files

In [162]:
%cp "/home/ubuntu/espnet/tools/sph2pipe_v2.5/sph2pipe" /home/ubuntu/project/notebook/
%cd /home/ubuntu/project/notebook/

/home/ubuntu/project/notebook


In [173]:
orig_wsj_data_root = "/home/ubuntu/espnet/egs2/wsj/asr1/data/"
datasets = os.listdir(wsj_data_root)
wsj_data_root = "/home/ubuntu/data/WSJ"

from tqdm import tqdm
for subset in datasets:
    if subset not in ['local', 'nlsyms.txt']:
        f = open(orig_wsj_data_root + subset + "/wav.scp", 'r')
        wavs = f.readlines()
        f.close()
        os.system(f"mkdir {wsj_data_root}/{subset}")
        print("number of waves = {} in {}".format(len(wavs), subset))
        print("start converting")
        for wav in tqdm(wavs):
            parts = wav.split(" ")
            file_path = parts[-2]
            name=file_path.split("/")[-1]
            save_file = wsj_data_root+"/"+subset+"/"+name+".wav"
            os.system(f"./sph2pipe -f rif {path} {save_file}")
        print(f"finish converting {subset}")

  3%|▎         | 10/330 [00:00<00:03, 96.72it/s]

number of waves = 330 in test_eval92_5k
start converting


  2%|▏         | 10/503 [00:00<00:04, 98.75it/s]

finish converting test_eval92_5k
number of waves = 503 in test_dev93
start converting


  3%|▎         | 10/333 [00:00<00:03, 99.50it/s]

...

finish converting dev_dt_20
number of waves = 513 in test_dev93_5k
start converting


  0%|          | 10/37416 [00:00<06:21, 98.00it/s]

finish converting test_dev93_5k
number of waves = 37416 in train_si284
start converting


100%|██████████| 37416/37416 [06:23<00:00, 97.55it/s]

finish converting train_si284


## manifest

In [16]:
%cd /home/ubuntu/fairseq
!python examples/wav2vec/wav2vec_manifest.py "/home/ubuntu/data/WSJ/train_si284" \
--dest "/home/ubuntu/project/manifest/train/" --ext "wav" --valid-percent 0

/home/ubuntu/fairseq


In [49]:
!head /home/ubuntu/project/manifest/train/train.tsv

/home/ubuntu/data/WSJ/train_si284
4asc031c.wv1.wav	207299
4a6c030h.wv1.wav	207299
019c020k.wv1.wav	207299
40nc020q.wv1.wav	207299
47tc020x.wv1.wav	207299
48sc0203.wv1.wav	207299
4a2c0309.wv1.wav	207299
01kc020d.wv1.wav	207299
4b1c020l.wv1.wav	207299


In [17]:
%cd /home/ubuntu/fairseq
!python examples/wav2vec/wav2vec_manifest.py "/home/ubuntu/data/WSJ/test_dev93/" \
--dest "/home/ubuntu/project/manifest/valid/" --ext "wav" --valid-percent 1

/home/ubuntu/fairseq


In [18]:
!mv /home/ubuntu/project/manifest/valid/valid.tsv /home/ubuntu/project/manifest/train/

## create labels

In [19]:
output_dir = "/home/ubuntu/project/manifest/train/"

In [57]:
# read transcriptions 
def generate_transcriptions_dict(transcriptions_path):
    all_words = ""
    with open(transcriptions_path, 'r') as trans:
        transcriptions = {}
        texts = trans.readlines()
        for text in texts:
            name, *words = text.strip("\n").split(" ")
            transcriptions[name] = " ".join(words)
            all_words=all_words+" ".join(words)
    return transcriptions, all_words

transcriptions_path = "/home/ubuntu/espnet/egs2/wsj/asr1/data/test_dev93/text"
valid_trans, all_words = generate_transcriptions_dict(transcriptions_path)

transcriptions_path = "/home/ubuntu/espnet/egs2/wsj/asr1/data/train_si284/text"
train_trans, all_words = generate_transcriptions_dict(transcriptions_path)

In [41]:
# calculate word frequency
from collections import Counter
wordcount = Counter(all_words)
word_count = wordcount.most_common(len(wordcount))
with open(output_dir + "dict.ltr.txt", 'w') as f:
    for word, cnt in word_count:
        if word == ' ':
            print("| {}".format(cnt), file=f)
        else:
            print(f"{word} {cnt}", file=f)

In [59]:
import os
def write_labels(tsv_path, output_dir, output_name, transcriptions):

    with open(tsv_path, "r") as tsv, \
    open(os.path.join(output_dir, output_name + ".ltr"), "w") as ltr_out, \
    open(os.path.join(output_dir, output_name + ".wrd"), "w") as wrd_out:
        root = next(tsv).strip()
        for line in tsv:
            line = line.strip()
            name = line.split(".")[0]
            if name in transcriptions:
                trans = transcriptions[name]
                print(trans, file=wrd_out)#    
                print(" ".join(trans.replace(" ", "|")) + " |",file=ltr_out)
            else:
                raise Exception("missing {} in transcriptions".format(name))
                
output_name = "train"
tsv = output_dir + "train.tsv"
write_labels(tsv, output_dir, output_name, transcriptions=train_trans)

output_name = "valid"
tsv = output_dir + "valid.tsv"
write_labels(tsv, output_dir, output_name, transcriptions=valid_trans)

In [1]:
ls /home/ubuntu/project/manifest/train

checkpoint_last.pt  nohup.out  tokens.txt  train.tsv  valid.ltr  valid.wrd
dict.ltr.txt        outputs/   train.ltr   train.wrd  valid.tsv


In [4]:
import soundfile
import os
train_dir = "/home/ubuntu/data/WSJ/train_si284/"
total_duration = 0
files = os.listdir(train_dir)
for f in files:
    sec = soundfile.info(train_dir+f).duration
    total_duration += sec

In [5]:
total_duration//3600

134.0

continue

2020-12-05 07:31:23 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 200, 'log_format': 'json', 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': True}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_port': -1, 'dev

In [1]:
pwd

'/home/ubuntu/project/notebook'

In [ ]:
!env HYDRA_FULL_ERROR=1 fairseq-hydra-train \
    task.data=/home/ubuntu/project/manifest/train/ \
    model.w2v_path=/home/ubuntu/project/model/wav2vec_small.pt \
    --config-dir /home/ubuntu/fairseq/examples/wav2vec/config/finetuning \
    --config-name base_100h

2020-12-05 08:04:51 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 200, 'log_format': 'json', 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': True}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_port': -1, 'dev

2020-12-05 08:04:58 | INFO | fairseq.trainer | detected shared parameter: w2v_encoder.w2v_model.feature_extractor.conv_layers.0.0.bias <- w2v_encoder.w2v_model.feature_extractor.conv_layers.1.0.bias
2020-12-05 08:04:58 | INFO | fairseq.trainer | detected shared parameter: w2v_encoder.w2v_model.feature_extractor.conv_layers.0.0.bias <- w2v_encoder.w2v_model.feature_extractor.conv_layers.2.0.bias
2020-12-05 08:04:58 | INFO | fairseq.trainer | detected shared parameter: w2v_encoder.w2v_model.feature_extractor.conv_layers.0.0.bias <- w2v_encoder.w2v_model.feature_extractor.conv_layers.3.0.bias
2020-12-05 08:04:58 | INFO | fairseq.trainer | detected shared parameter: w2v_encoder.w2v_model.feature_extractor.conv_layers.0.0.bias <- w2v_encoder.w2v_model.feature_extractor.conv_layers.4.0.bias
2020-12-05 08:04:58 | INFO | fairseq.trainer | detected shared parameter: w2v_encoder.w2v_model.feature_extractor.conv_layers.0.0.bias <- w2v_encoder.w2v_model.feature_extractor.conv_layers.5.0.bias
2020-

2020-12-05 08:05:39 | WARNING | fairseq.trainer | OOM: Ran out of memory with exception: CUDA out of memory. Tried to allocate 648.00 MiB (GPU 0; 14.76 GiB total capacity; 3.35 GiB already allocated; 58.75 MiB free; 6.17 GiB reserved in total by PyTorch)
2020-12-05 08:05:39 | WARNING | fairseq.trainer | |===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 3            |        cudaMalloc retries: 6         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    2786 MB |    5819 MB |  426174 MB |  423387 MB |
|       from large pool |    2782 MB |    5815 MB |  426019 MB |  42323

|       from large pool |     153    |     329    |   32491    |   32338    |
|       from small pool |     244    |     318    |   22053    |   21809    |
|---------------------------------------------------------------------------|
| Active allocs         |     397    |     646    |   54544    |   54147    |
|       from large pool |     153    |     329    |   32491    |   32338    |
|       from small pool |     244    |     318    |   22053    |   21809    |
|---------------------------------------------------------------------------|
| GPU reserved segments |      36    |      55    |     190    |     154    |
|       from large pool |      33    |      52    |     179    |     146    |
|       from small pool |       3    |       4    |      11    |       8    |
|---------------------------------------------------------------------------|
| Non-releasable allocs |      32    |      49    |   25675    |   25643    |
|       from large pool |      26    |      36    |   18751    |

2020-12-05 08:06:31 | WARNING | fairseq.trainer | OOM: Ran out of memory with exception: CUDA out of memory. Tried to allocate 648.00 MiB (GPU 0; 14.76 GiB total capacity; 3.37 GiB already allocated; 96.75 MiB free; 6.14 GiB reserved in total by PyTorch)
2020-12-05 08:06:31 | WARNING | fairseq.trainer | |===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 9            |        cudaMalloc retries: 17        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    2802 MB |    5822 MB |    1073 GB |    1070 GB |
|       from large pool |    2798 MB |    5817 MB |    1073 GB |    107